## **Variational classifier**

## **Fitting the parity function**

Import PennyLane, the PennyLane-provided version of NumPy, and an optimizer.



In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

This sets up a quantum device using PennyLane's built-in state vector simulator, default.qubit. By initializing this simulator, we create an environment where quantum circuits can be executed classically, mimicking the behavior of a real quantum computer. This device handles the application of quantum gates and measurement of qubit states, providing a foundation for running and optimizing quantum circuits in our machine learning workflow. The default.qubit simulator is ideal for development and testing, allowing us to fit functions like the parity function without needing access to actual quantum hardware.

In [ ]:
dev = qml.device("default.qubit")

This function defines a single layer of a quantum circuit. Each layer consists of rotation gates (`qml.Rot`) applied to four qubits, parameterized by `layer_weights`, followed by entangling CNOT gates between specific pairs of qubits. The rotation gates manipulate the states of individual qubits, while the CNOT gates create entanglement between them, allowing for complex correlations. This combination forms a versatile building block for a quantum neural network, enabling the circuit to represent intricate quantum states necessary for sophisticated quantum computations and function approximations.

In [ ]:
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

### Step: State Preparation



#### Explanation:
This function prepares the initial quantum state of the system by setting it to a specific basis state. The function `state_preparation(x)` takes an input `x`, which is typically a binary vector representing the desired basis state. The `qml.BasisState` operation initializes the quantum state of the qubits to this specific state. The argument `x` is applied to the qubits indexed by `[0, 1, 2, 3]`, meaning that `x` must be a binary array of length 4. For example, if `x = [1, 0, 1, 0]`, the quantum state will be prepared in the basis state \(|1010\rangle\). This step is crucial for quantum algorithms that require the system to start in a specific known state, ensuring that the subsequent operations and transformations applied by the quantum circuit are based on a well-defined initial condition. This allows for consistent and accurate execution of quantum computations and measurements.

In [ ]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])


This code defines a quantum circuit for a variational quantum algorithm using PennyLane. The `@qml.qnode(dev)` decorator indicates that this function is a quantum node (QNode) that will be executed on the specified quantum device `dev`. The function `circuit(weights, x)` takes `weights` and `x` as inputs. First, it prepares the quantum state using `state_preparation(x)`, which initializes the qubits to a specific basis state defined by `x`. Then, it iterates through `weights`, a list of parameters, applying the `layer` function to construct multiple layers of the quantum circuit, each parameterized by `layer_weights`. Each layer consists of rotation and entangling gates that transform the quantum state. Finally, the circuit returns the expectation value of the Pauli-Z operator on the first qubit (`qml.expval(qml.PauliZ(0))`), which is a common measurement used to extract information about the quantum state. This expectation value serves as the output of the quantum circuit, which can be used for tasks like classification or regression in quantum machine learning.

In [ ]:
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

### Step: Defining the Variational Classifier

This function defines a variational classifier, a type of quantum machine learning model. The `variational_classifier(weights, bias, x)` function takes three inputs: `weights`, `bias`, and `x`. The `weights` are the parameters for the quantum circuit layers, and `x` is the input data that determines the initial state of the quantum system. The function calls the `circuit(weights, x)`, which prepares the quantum state, applies the layered quantum gates, and measures the expectation value of the Pauli-Z operator on the first qubit. This expectation value represents the output of the quantum circuit based on the given parameters and input state. The function then adds a classical `bias` term to this quantum output. This bias term allows for more flexibility in the model, enabling it to better fit the training data. The result, `circuit(weights, x) + bias`, is the output of the variational classifier, which can be used for tasks such as binary classification, where the output would typically be further processed to make predictions. This combination of quantum and classical components forms a hybrid model, leveraging the strengths of both quantum computing and classical machine learning.

In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

**Step: Calculate the loss**

This function calculates the square loss, which is a common metric for evaluating the performance of regression models. The square_loss(labels, predictions) function takes labels, the true values, and predictions, the values predicted by the model. The qml.math.stack function is used to ensure that the predictions are in a compatible format for array operations. The function computes the mean squared error between the labels and predictions by subtracting the arrays element-wise, squaring the differences, and then taking the mean of these squared differences. This loss value quantifies how closely the model's predictions match the true labels, with lower values indicating better performance.

In [ ]:
def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

**Step: Defining the Accuracy Metric**

This function calculates the accuracy of the model's predictions. The accuracy(labels, predictions) function takes two inputs: labels, which are the true values, and predictions, which are the values predicted by the model. It computes the number of correct predictions by checking if the absolute difference between each label and its corresponding prediction is less than a small threshold (1e-5). This threshold accounts for potential numerical precision issues. The sum of these correct predictions is then divided by the total number of labels to obtain the accuracy, which is a measure of how many predictions are correct relative to the total number of predictions. The function returns this accuracy value, with a range between 0 (no correct predictions) and 1 (all predictions correct).

In [ ]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

**Step: Defining the Cost Function**

This function defines the cost function, which measures the performance of the variational classifier. The cost(weights, bias, X, Y) function takes four inputs: weights and bias, which are the parameters of the quantum model; X, the input data; and Y, the true labels. It first generates predictions for each input in X by applying the variational_classifier(weights, bias, x) to each input x. These predictions are collected into a list. The function then calculates the square loss between the true labels Y and the model's predictions using the square_loss(Y, predictions) function.

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

**Optimization**

**Step: Loading and Preprocessing Data**

The data is loaded into the variable data. The input features are extracted into the array X by selecting all columns except the last one (data[:, :-1]), while the labels are extracted into the array Y by selecting the last column (data[:, -1]). The labels Y are then transformed from the range {0, 1} to {-1, 1} using the expression Y = Y * 2 - 1, which is a common preprocessing step for certain types of classification problems, particularly those involving quantum circuits.

In [ ]:
data = np.loadtxt("variational_classifier/data/parity_train.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

### Step: Initializing the Model Parameters


This code initializes the model parameters for the variational quantum classifier. By setting the random seed to 0, it ensures reproducibility. The weights are initialized with small random values close to zero, suitable for 2 layers and 4 qubits, and are flagged for gradient tracking. The bias is initialized to 0.0 and also set for gradient tracking. This setup is crucial for training the model effectively.

In [ ]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)


**Step: Training the Model**


The model is trained using the Nesterov Momentum Optimizer with a learning rate of 0.5. In each of the 100 iterations, a batch of 5 random data points is selected to update the weights and bias. The accuracy and cost are computed for each iteration, and these metrics are printed to monitor the training progress. This approach helps in efficiently optimizing the model parameters using stochastic gradient descent.

In [ ]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5
weights = weights_init
bias = bias_init
for it in range(100):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

**Iris classification**

### Step: State Preparation for Iris Dataset


#### Explanation:
The `get_angles` function translates input data `x` from the Iris dataset into a set of rotation angles for state preparation. These angles are calculated using trigonometric functions based on the components of `x`. The `state_preparation` function then uses these angles to prepare the quantum state by applying a sequence of RY rotations and CNOT gates. This setup allows the quantum circuit to encode the classical input data into a quantum state efficiently, facilitating subsequent quantum computations. The process ensures that the input features are represented as quantum states that can be manipulated and measured within the quantum algorithm.

In [ ]:
def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(np.linalg.norm(x[2:]) / np.linalg.norm(x))

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])


def state_preparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

### Step: Testing State Preparation

This step tests the state preparation function with a sample input vector `x` from the Iris dataset. The `get_angles` function converts `x` into rotation angles, which are then used in the `state_preparation` function within a quantum node (`test`) to prepare the quantum state. The resulting quantum state is retrieved and printed, along with the input vector and the computed angles. This verification step ensures that the state preparation is correctly encoding the input data into the desired quantum state.

In [ ]:
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0], requires_grad=False)
ang = get_angles(x)


@qml.qnode(dev)
def test(angles):
    state_preparation(angles)

    return qml.state()


state = test(ang)

print("x               : ", np.round(x, 6))
print("angles          : ", np.round(ang, 6))
print("amplitude vector: ", np.round(np.real(state), 6))

### Step: Defining the Quantum Circuit Layer and Cost Function

This step defines a single layer of the quantum circuit and the cost function for the variational classifier. The `layer` function applies rotation gates to each qubit and a CNOT gate to entangle them. The `cost` function computes the loss between the model's predictions and the true labels by first preparing the state using the variational classifier with the input data `X` (transposed for correct indexing in `state_preparation`). It then calculates the square loss to evaluate the performance of the model, which will be used to guide the optimization during training.

In [ ]:
def layer(layer_weights):
    for wire in range(2):
        qml.Rot(*layer_weights[wire], wires=wire)
    qml.CNOT(wires=[0, 1])


def cost(weights, bias, X, Y):
    # Transpose the batch of input data in order to make the indexing
    # in state_preparation work
    predictions = variational_classifier(weights, bias, X.T)
    return square_loss(Y, predictions)

### Step: Preprocessing the Iris Dataset

This step involves preparing the Iris dataset for use in the quantum classifier. The input data `X` is loaded and padded to a length of 4 with constant values. The padded data is then normalized to ensure uniform scaling. The `get_angles` function is applied to each normalized input to convert them into rotation angles for the quantum state preparation. Finally, the labels `Y` are extracted from the dataset. This preprocessing is essential to convert the raw data into a form suitable for quantum computation.

In [ ]:
data = np.loadtxt("variational_classifier/data/iris_classes1and2_scaled.txt")
X = data[:, 0:2]
print(f"First X sample (original)  : {X[0]}")

# pad the vectors to size 2^2=4 with constant values
padding = np.ones((len(X), 2)) * 0.1
X_pad = np.c_[X, padding]
print(f"First X sample (padded)    : {X_pad[0]}")

# normalize each input
normalization = np.sqrt(np.sum(X_pad**2, -1))
X_norm = (X_pad.T / normalization).T
print(f"First X sample (normalized): {X_norm[0]}")

# the angles for state preparation are the features
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)
print(f"First features sample      : {features[0]}")

Y = data[:, -1]

### Step: Visualizing the Data

This step involves visualizing the Iris dataset at different stages of preprocessing. The first plot shows the original data with class labels. The second plot displays the padded and normalized data, highlighting how the data distribution changes after normalization. The third plot visualizes the feature vectors derived from the normalized data using the `get_angles` function, showing how the features are mapped into a higher-dimensional space for quantum state preparation. These visualizations help in understanding the transformations applied to the data and their impact on its distribution.

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(X[:, 0][Y == 1], X[:, 1][Y == 1], c="b", marker="o", ec="k")
plt.scatter(X[:, 0][Y == -1], X[:, 1][Y == -1], c="r", marker="o", ec="k")
plt.title("Original data")
plt.show()

plt.figure()
dim1 = 0
dim2 = 1
plt.scatter(X_norm[:, dim1][Y == 1], X_norm[:, dim2][Y == 1], c="b", marker="o", ec="k")
plt.scatter(X_norm[:, dim1][Y == -1], X_norm[:, dim2][Y == -1], c="r", marker="o", ec="k")
plt.title(f"Padded and normalised data (dims {dim1} and {dim2})")
plt.show()

plt.figure()
dim1 = 0
dim2 = 3
plt.scatter(features[:, dim1][Y == 1], features[:, dim2][Y == 1], c="b", marker="o", ec="k")
plt.scatter(features[:, dim1][Y == -1], features[:, dim2][Y == -1], c="r", marker="o", ec="k")
plt.title(f"Feature vectors (dims {dim1} and {dim2})")
plt.show()

### Step: Splitting the Data into Training and Validation Sets

This step splits the dataset into training and validation sets. The total number of data points is determined, and 75% of the data is allocated for training, with the remaining 25% used for validation. The data indices are randomly permuted to ensure a random split. The features and labels are then divided into training (`feats_train`, `Y_train`) and validation sets (`feats_val`, `Y_val`). Additionally, the original input features (`X_train` and `X_val`) are split similarly for later plotting. This split is essential for training the model and evaluating its performance on unseen data.

In [ ]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

### Optimization
### Step: Training and Evaluating the Model

This step involves training the variational quantum classifier and evaluating its performance. The weights and bias are initialized and updated using the Nesterov Momentum Optimizer over 60 iterations. In each iteration, a random batch of training data is selected to update the model parameters. The accuracy of the model is computed on both the training and validation sets. The progress is printed every two iterations, showing the cost, training accuracy, and validation accuracy. This iterative process aims to optimize the model parameters and monitor its performance on both seen and unseen data.

In [ ]:
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = np.sign(variational_classifier(weights, bias, feats_train.T))
    predictions_val = np.sign(variational_classifier(weights, bias, feats_val.T))

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    if (it + 1) % 2 == 0:
        _cost = cost(weights, bias, features, Y)
        print(
            f"Iter: {it + 1:5d} | Cost: {_cost:0.7f} | "
            f"Acc train: {acc_train:0.7f} | Acc validation: {acc_val:0.7f}"
        )

Plot the continuous output of the variational classifier for the first two dimensions of the Iris data set

In [ ]:
plt.figure()
cm = plt.cm.RdBu

# make data for decision regions
xx, yy = np.meshgrid(np.linspace(0.0, 1.5, 30), np.linspace(0.0, 1.5, 30))
X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# preprocess grid points like data inputs above
padding = 0.1 * np.ones((len(X_grid), 2))
X_grid = np.c_[X_grid, padding]  # pad each input
normalization = np.sqrt(np.sum(X_grid**2, -1))
X_grid = (X_grid.T / normalization).T  # normalize each input
features_grid = np.array([get_angles(x) for x in X_grid])  # angles are new features
predictions_grid = variational_classifier(weights, bias, features_grid.T)
Z = np.reshape(predictions_grid, xx.shape)

# plot decision regions
levels = np.arange(-1, 1.1, 0.1)
cnt = plt.contourf(xx, yy, Z, levels=levels, cmap=cm, alpha=0.8, extend="both")
plt.contour(xx, yy, Z, levels=[0.0], colors=("black",), linestyles=("--",), linewidths=(0.8,))
plt.colorbar(cnt, ticks=[-1, 0, 1])

# plot data
for color, label in zip(["b", "r"], [1, -1]):
    plot_x = X_train[:, 0][Y_train == label]
    plot_y = X_train[:, 1][Y_train == label]
    plt.scatter(plot_x, plot_y, c=color, marker="o", ec="k", label=f"class {label} train")
    plot_x = (X_val[:, 0][Y_val == label],)
    plot_y = (X_val[:, 1][Y_val == label],)
    plt.scatter(plot_x, plot_y, c=color, marker="^", ec="k", label=f"class {label} validation")

plt.legend()
plt.show()